## PageRank

Developed by Larry Page and the backbone of Google, PageRank was initially applied to web pages and the links between them. In some cases, PageRank can also be a good ratings system in sports. I have found it does well in One vs. Many applications, like a golf tournament. I've added a way to implement time decay with it as well. Uniquely among rating systems, it also can be used to give a measure of entropy and uncertainty. There is a major limitation in my implementation however. In most cases, you'll want to also adjust for things like home/away and weather. PageRank doesn't have a great way to deal with that, other than using other computational methods to estimate the effects and then applying weights to your PageRank data. Yet still, I've found there are some niche applications where there are little other effects, like esports and already adjusted golf strokes gained data, where PageRank is very competitive. 

### Advantages
- Simplicity, intuitiveness 
- Computational Speed
- Can measure entropy
- Can account for score difference
- Can handle any number of opponents
- Can scale as big as you need

### Disadvantages
- Need to keep number of players/teams constant or else ratings change scale
- No innate way to adjust for meta factors
- Not robust to param choices, and defaults aren't obvious because they depend on scale of scoring and number of teams
- Need separate logic for differences and totals

In [6]:

import os

import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.sparse import csr_matrix
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression


In [37]:
class Optimizer:
    def __init__(self):
        pass

    def optimize(self):
        raise NotImplementedError("Subclasses must implement the optimize method.")
    
class PageRankOptimizer(Optimizer):
    def __init__(self, protag_col='team', antag_col='opponent', stat_col='score', static_protag_number=200, method='pagerank', time_decay=True):
        super().__init__()
        self.protag_col = protag_col
        self.antag_col = antag_col
        assert(protag_col != antag_col), "Protagonist and Antagonist columns must be different"
        self.stat_col = stat_col
        self.time_decay = time_decay
        self.static_protag_number = static_protag_number
        assert(method in ['pagerank', 'sparse', 'power_iteration']), "Method not recognized"
        self.method = method

    def load_data(self, data, path=None):
        if path is not None:
            self.data = pd.read_csv(path)
        else:
            self.data = data
        self.preprocess_data()
        
    def preprocess_data(self):
        assert(self.protag_col in self.data.columns), "Protagonist column not found in data"
        assert(self.antag_col in self.data.columns), "Antagonist column not found in data"
        assert(self.stat_col in self.data.columns), "Stat column not found in data"
        assert('num_opponents' in self.data.columns), "Number of opponents column not found in data"

        # Convert date column to datetime if needed
        if isinstance(self.data['date'].iloc[0], str):
            self.data['date'] = pd.to_datetime(self.data['date'])

        # Sort data by date
        self.data = self.data.sort_values('date').reset_index(drop=True)

        assert(len(self.data)>200), "Not enough data to optimize"

    def pagerank(self, A, l2):

        num_protags = A.shape[0]
        b = np.random.rand(num_protags,1)
        b = b/np.linalg.norm(b, 1)
        for i in range(50):
            b = l2 * A@b + (1-l2)/num_protags

        return b

    def pagerank_sparse(self, A, l2):
        
        A = csr_matrix(A)
        num_protags = A.shape[0]
        b = np.random.rand(num_protags,1)
        b = b/np.linalg.norm(b, 1)
        for i in range(50):
            b = l2 * A@b + (1-l2)/num_protags

        return b

    def pagerank_power_iteration(self, A, l2, b, p_iter=25):
        num_protags = A.shape[0]
        for i in range(p_iter):
            for j in range(num_protags):
                b[j] = (1-l2)/num_protags+l2*np.sum(A[k][j]*b[k] for k in range(num_protags))
        return A, b
    
    def run_history(self, halflife, l2, record_dates, return_entropy=False):

        ratings = []

        decay = np.exp(-np.log(2)/halflife)
        A = np.zeros((self.static_protag_number, self.static_protag_number))
        A_inv = np.zeros((self.static_protag_number, self.static_protag_number))
        last_date = self.data['date'].iloc[0]

        if self.method == 'power_iteration':
            # need to initialize ratings
            rtgs = np.random.rand(self.static_protag_number, 1)
            rtgs_inv = np.random.rand(self.static_protag_number, 1)
        
        for date, day_df in tqdm(self.data.groupby(['date']), total=len(self.data['date'].unique())):
            
            ## decay ratings matrices
            days_ago = (date-last_date).days
            A = A * decay**days_ago
            A_inv = A_inv * decay**days_ago
            last_date = date

            ## first update ratings
            if date in record_dates:
                # if date requested, calculate ratings
                if self.method == 'pagerank':
                    rtgs = self.pagerank(A, l2)
                    rtgs_inv = self.pagerank(A_inv, l2)
                elif self.method == 'sparse':
                    rtgs = self.pagerank_sparse(A, l2)
                    rtgs_inv = self.pagerank_sparse(A_inv, l2)
                elif self.method == 'power_iteration':
                    A, rtgs = self.pagerank_power_iteration(A, l2, rtgs)
                    A_inv, rtgs_inv = self.pagerank_power_iteration(A_inv, l2, rtgs_inv)
                rtg_df = pd.DataFrame({self.protag_col: [protag_map_inv[i] for i in range(self.static_protag_number)], 'rating': rtgs.reshape(-1)})
                inv_rtg_df = pd.Series(rtgs_inv.reshape(-1), name='inv_rating')
                rtg_df = pd.concat([rtg_df, inv_rtg_df], axis=1)
                to_append = day_df.merge(rtg_df, on=self.protag_col)
                rtg_df = rtg_df.rename(columns={self.protag_col: self.antag_col, 'rating': 'antag_rtg', 'inv_rating': 'inv_antag_rtg'})
                to_append = to_append.merge(rtg_df, on=self.antag_col)
                ratings.append(to_append)

            # then use results to update rating matrix
            protags = self.data[self.protag_col].value_counts()[:self.static_protag_number]
            protag_map = {protag: i for i, protag in enumerate(protags.index)}
            protag_map_inv = {i: protag for i, protag in enumerate(protags.index)}
            day_df = day_df[day_df[self.protag_col].isin(protags.index)]
            day_df = day_df[day_df[self.antag_col].isin(protags.index)]
            day_df['protag_idx'] = day_df[self.protag_col].map(protag_map)
            day_df['antag_idx'] = day_df[self.antag_col].map(protag_map)
            ## positive only is more stable
            day_df['stat'] = pd.DataFrame({
                self.stat_col: day_df[self.stat_col],
                'zero_max': np.zeros(len(day_df))
            }).max(axis=1)
            day_df['stat'] /= day_df['num_opponents']
            A[day_df['protag_idx'], day_df['antag_idx']] += day_df[self.stat_col]
            A_inv[day_df['antag_idx'], day_df['protag_idx']] += day_df[self.stat_col]


        ratings = pd.concat(ratings, axis=0).reset_index(drop=True)
        ratings['rating'] = ratings['rating'] - ratings['inv_rating']
        ratings['opp_rating'] = ratings['antag_rtg'] - ratings['inv_antag_rtg']
        ratings = ratings.drop(columns=['inv_rating', 'inv_antag_rtg', 'antag_rtg'])
        return ratings

    
    def run_time_opt(self, init_points=10, n_iter=30, num_test_dates=20, halflife_bounds=(10, 800), l2_bounds=(1e-9, 10)):

        # Select random test dates
        unique_dates = self.data['date'].unique()
        assert(len(self.data)>200), "Not enough data to optimize"
        ## don't take from the first 10 or so dates
        unique_dates = sorted(unique_dates)[10:]

        test_dates = np.random.choice(unique_dates, size=num_test_dates, replace=False)
        num_dates = len(unique_dates)

        pbounds = {'halflife': halflife_bounds, 'l2': l2_bounds}

        def time_bayes_objective(halflife, l2):
            
            ratings = self.run_history(halflife==halflife, l2=l2, record_dates=test_dates)
            X = ratings[['rating','opp_rating']]
            y = ratings[self.stat_col]
            lr = LinearRegression()
            lr.fit(X, y)
            preds = lr.predict(X)
            mse = mean_squared_error(y, preds)

            if ratings['rating'].max() > 1e10:
                return -1e10
            return -mse

        # Initialize the Bayesian Optimization object
        optimizer = BayesianOptimization(f=time_bayes_objective, pbounds=pbounds, random_state=17)

        # Perform the optimization
        optimizer.maximize(init_points=init_points, n_iter=n_iter)
        
        # Get the best parameters and correlation
        best_params = optimizer.max['params']
        print(best_params)
        best_halflife = best_params['halflife']
        best_l2 = best_params['l2']
        best_mse = -optimizer.max['target']

        return best_halflife, best_l2, best_mse
    
    def get_ratings_for_dates(self, dates, l2, halflife=250, stat_type='diff', return_entropy=False):
        dates = pd.to_datetime(dates)
        dates = sorted(dates)

        ## pagerank it is easier to just run entire history
        return self.run_history(halflife, l2, dates, return_entropy=return_entropy)



In [38]:
halflife = 250
np.exp(-np.log(2)/halflife)


0.9972312513520695

In [39]:

data_path = '../data'
sample_data = pd.read_csv(os.path.join(data_path, 'testing', 'ncaam_sample_data.csv'))
sample_data['score_diff'] = sample_data['team_score'] - sample_data['opp_score']
sample_data['date'] = pd.to_datetime(sample_data['date'])
sample_data = sample_data.sort_values('date')
sample_data = sample_data.loc[sample_data['date'] > '2019-10-01'].reset_index(drop=True)
sample_data['num_opponents'] = 1
sample_data.head()


,season,team_score,opp_score,is_home,numot,team_fgm,team_fga,team_fgm3,team_fga3,team_ftm,...,opp_ast,opp_to,opp_stl,opp_blk,opp_pf,team_name,opp_name,date,score_diff,num_opponents
0,2020,81,80,-1,0,29,68,8,26,15,...,10,19,10,5,18,Penn,Alabama,2019-11-05,1,1
1,2020,84,46,1,0,29,58,12,31,14,...,8,19,7,1,22,Penn St,MD E Shore,2019-11-05,38,1
2,2020,71,87,-1,0,26,70,10,29,9,...,12,12,4,1,13,Pepperdine,California,2019-11-05,-16,1
3,2020,74,65,1,0,26,60,5,21,17,...,12,19,7,3,19,SMU,Jacksonville St,2019-11-05,9,1
4,2020,67,94,-1,0,27,57,7,22,6,...,13,8,10,5,15,Princeton,Duquesne,2019-11-05,-27,1


In [40]:
PO = PageRankOptimizer(
    protag_col='team_name', 
    antag_col='opp_name', 
    stat_col='score_diff', 
    static_protag_number=sample_data['team_name'].nunique()
)

PO.load_data(sample_data)


In [41]:

rtgs_history = PO.get_ratings_for_dates(sample_data.date.unique()[-100:], l2=0.8/sample_data.team_name.nunique(), halflife=250, stat_type='diff', return_entropy=False)



  0%|          | 0/381 [00:00<?, ?it/s]c:\Users\Blake\anaconda3\Lib\site-packages\tqdm\std.py:1178: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
100%|██████████| 381/381 [00:02<00:00, 151.07it/s]


In [42]:

halflife, l2, mse = PO.run_time_opt(init_points=10, n_iter=30, num_test_dates=60, halflife_bounds=(10, 800), l2_bounds=(1e-9, 1e-2))


|   iter    |  target   | halflife  |    l2     |
-------------------------------------------------


  0%|          | 0/381 [00:00<?, ?it/s]c:\Users\Blake\anaconda3\Lib\site-packages\tqdm\std.py:1178: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
100%|██████████| 381/381 [00:02<00:00, 185.28it/s]


| 1         | -1e+10    | 242.8     | 5.306     |


  0%|          | 0/381 [00:00<?, ?it/s]c:\Users\Blake\anaconda3\Lib\site-packages\tqdm\std.py:1178: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
100%|██████████| 381/381 [00:02<00:00, 171.93it/s]


| 2         | -1e+10    | 161.3     | 0.679     |


...

| 14        | -1e+10    | 11.46     | 0.1539    |


  0%|          | 0/381 [00:00<?, ?it/s]c:\Users\Blake\anaconda3\Lib\site-packages\tqdm\std.py:1178: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
100%|██████████| 381/381 [00:02<00:00, 179.91it/s]


| 15        | -1e+10    | 765.8     | 5.588     |


  0%|          | 0/381 [00:00<?, ?it/s]c:\Users\Blake\anaconda3\Lib\site-packages\tqdm\std.py:1178: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
  2%|▏         | 9/381 [00:00<00:01, 188.48it/s]


KeyboardInterrupt: 

In [ ]:



rtgs_history = PO.get_ratings_for_dates(sample_data.date.unique()[-100:], l2=l2, halflife=halflife, stat_type='diff', return_entropy=False)



  0%|          | 0/381 [00:00<?, ?it/s]c:\Users\Blake\anaconda3\Lib\site-packages\tqdm\std.py:1178: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for obj in iterable:
100%|██████████| 381/381 [00:02<00:00, 149.92it/s]


In [28]:
test_ratings = rtgs_history.groupby(['team_name'])[['rating']].last().copy().reset_index()
test_ratings.sort_values(by=['rating'], ascending=False).head(25)

,team_name,rating
20,Baylor,6.668578e+134
149,Louisiana Tech,5.514692e+134
0,Abilene Chr,4.963387e+134
280,St Mary's CA,4.660335e+134
254,SMU,4.614773e+134
156,MS Valley St,4.324825e+134
12,Ark Pine Bluff,4.077192e+134
115,Houston Chr,3.699619e+134
279,St Louis,3.234831e+134
106,Grand Canyon,3.139403e+134
